In [8]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [2]:
# Connect Colab to Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Unzip training data, run once
!unzip drive/MyDrive/data/train.zip -d drive/MyDrive/data/

In [ ]:
# Unzip validation data, run once
!unzip drive/MyDrive/data/validation.zip -d drive/MyDrive/data/

In [11]:
# This code chunk puts all image paths and their labels in a dataframe
train_paths = []
train_labels = []
validation_paths = []
validation_labels = []
troot = 'drive/MyDrive/data/train-cropped'
vroot = 'drive/MyDrive/data/validation-cropped'

for f in os.listdir(troot):
  tpath = os.path.join(troot, f)
  tlabel = f[-10:-8]
  train_paths.append(tpath)
  train_labels.append(tlabel)

for f in os.listdir(vroot):
  vpath = os.path.join(vroot, f)
  vlabel = f[-10:-8]
  validation_paths.append(vpath)
  validation_labels.append(vlabel)

train_images = pd.DataFrame({'path': train_paths,
                             'label': train_labels})
validation_images = pd.DataFrame({'path': validation_paths,
                                  'label': validation_labels})